In [ ]:
import pathlib
import re

import numpy as np

In [ ]:
root = pathlib.Path(r'D:\iComLogFiles')

holding_dir = root.joinpath('holding')
processing_dir = root.joinpath('processing')

In [ ]:
ip = '192.168.100.200'

In [ ]:
files_to_process = sorted(list(processing_dir.glob(f'{ip}*.txt')))
files_to_process

In [ ]:
def parse_datetime(data):
    pattern = re.compile(b'(\d+-\d+-\d\d)(\d\d:\d+:\d+)')
    datetime = pattern.findall(data)
    datetime = np.array(datetime).astype(str)
    date_val = datetime[0][0]
    time_val = datetime[0][1]
    utc_datetime_str = "{}T{}".format(date_val, time_val)
    filename_tag = re.sub('[-|:]', '',utc_datetime_str)
    
    return filename_tag

In [ ]:
def initial_results_parse(data):
    pattern = re.compile(b'\x00\x00\x00([a-zA-Z0-9 \.-]+)')

    results = pattern.findall(data)
    results = np.array(results).astype(str)
    
    return results

In [ ]:
def pull_header(tag, length, results_dict, results_scrape):
    index = np.where(results_scrape == tag)[0]

    for i, ref in enumerate(index):
        asymx = results_scrape[ref+1:ref+length+1]
        asymx = np.array(asymx).astype(float)
        asymx[(asymx == -32767) | (asymx == 32767)] = None
        results_dict["{}-{}".format(tag, i)] = asymx

    for ref in index[-1::-1]:
        results_scrape = np.delete(results_scrape, np.arange(ref,ref+length+1))
        
    return results_dict, results_scrape

In [ ]:
def organise_by_tags(results):
    results_dict = dict()
    results_scrape = results.copy()

    results_dict, results_scrape = pull_header('ASYMX', 2, results_dict, results_scrape)
    results_dict, results_scrape = pull_header('ASYMY', 2, results_dict, results_scrape)
    results_dict, results_scrape = pull_header('MLCX', 160, results_dict, results_scrape)

    pattern = re.compile('[a-zA-Z][a-zA-Z0-9 -]+')

    alpha_numeric = np.array([
        pattern.match(value)
        for value in results_scrape
    ]).astype(bool)
    results_dict["Text-tags"] = results_scrape[alpha_numeric].tolist()
    results_scrape = np.delete(results_scrape, np.where(alpha_numeric)[0])

    left_overs = results_scrape.astype(str)
    left_overs[(left_overs == '-32767') | (left_overs == '32767')] = None

    results_dict["Left-overs"] = left_overs.tolist()
    
    return results_dict

In [ ]:
data_left_over = b''

for path in files_to_process:
    print('new_file')
    with open(path, 'rb') as a_file:
        data = data_left_over + a_file.read()
        
    control_point_split = data.split(b'\x00\x00\x04\x04')
    
    
    
    for control_point in control_point_split[:-1]:
        print(len(control_point))
        print(len(control_point.split(b'\n')[-1]))
#         if len(control_point) != 8047:
#             print(control_point[0:10])
#             print(control_point[-10::])
        
    data_left_over = control_point_split[-1]
#         results = initial_results_parse(control_point_split[1])
#         results_dict = organise_by_tags(results)
        
#         if not 'Settings Not Confirmed' in results_dict['Text-tags']:
#             print(results_dict['Text-tags'])